In [9]:
import pandas as pd

# 시도명 매핑 딕셔너리 (변동 없음)
sido_map = {
    '서울': '서울특별시', '부산': '부산광역시', '대구': '대구광역시',
    '광주': '광주광역시', '울산': '울산광역시', '인천': '인천광역시',
    '대전': '대전광역시',
    
    # 도 단위 및 기타 지역 매핑
    '북춘천': '강원도', '북강릉': '강원도', '울릉도': '경상북도',
    '안동': '경상북도', '포항': '경상북도', '창원': '경상남도',
    '수원': '경기도', '청주': '충청북도', '전주': '전라북도',
    '목포': '전라남도', '여수': '전라남도', '흑산도': '전라남도',
    '홍성': '충청남도', '백령도': '인천광역시', '서귀포': '제주도',
    '제주': '제주도',
    
    # 기타 추가
    '속초': '강원도', '철원': '강원도', '동두천': '경기도', '파주': '경기도',
    '대관령': '강원도', '춘천': '강원도', '강릉': '강원도', '동해': '강원도',
    '원주': '강원도', '영월': '강원도', '충주': '충청북도', '서산': '충청남도',
    '울진': '경상북도', '추풍령': '경상북도', '상주': '경상북도',
    '군산': '전라북도', '통영': '경상남도', '완도': '전라남도', '고창': '전라북도',
    '순천': '전라남도', '진도(첨찰산)': '전라남도', '서청주': '충청북도',
    '고산': '제주도', '성산': '제주도', '진주': '경상남도',
    '강화': '인천광역시', '양평': '경기도', '이천': '경기도',
    '인제': '강원도', '홍천': '강원도', '태백': '강원도', '정선군': '강원도',
    '제천': '충청북도', '보은': '충청북도', '천안': '충청남도',
    '보령': '충청남도', '부여': '충청남도', '금산': '충청남도',
    '세종': '세종특별자치시', '부안': '전라북도', '임실': '전라북도',
    '정읍': '전라북도', '남원': '전라북도', '장수': '전라북도',
    '고창군': '전라북도', '영광군': '전라남도', '김해시': '경상남도',
    '순창군': '전라북도', '북창원': '경상남도', '양산시': '경상남도',
    '보성군': '전라남도', '강진군': '전라남도', '장흥': '전라남도',
    '해남': '전라남도', '고흥': '전라남도', '의령군': '경상남도',
    '함양군': '경상남도', '광양시': '전라남도', '진도군': '전라남도',
    '봉화': '경상북도', '영주': '경상북도', '문경': '경상북도',
    '청송군': '경상북도', '영덕': '경상북도', '의성': '경상북도',
    '구미': '경상북도', '영천': '경상북도', '경주시': '경상북도',
    '거창': '경상남도', '합천': '경상남도', '밀양': '경상남도',
    '산청': '경상남도', '거제': '경상남도', '남해': '경상남도',
    '북부산': '부산광역시'
}

# 1. 로드 및 시도명 매핑
# '일별_기온_강수량_일조15~24.csv' 파일을 실제 경로에 맞게 로드하세요.
try:
    df_temp = pd.read_csv('일별_기온_강수량_일조15~24.csv')
except FileNotFoundError:
    print("오류: '일별_기온_강수량_일조15~24.csv' 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
    exit()

df_temp['시도명'] = df_temp['지점명'].map(sido_map)

# 2. 날짜 필터링 (생육기) - 3월 15일 ~ 9월 10일
df_temp['일시'] = pd.to_datetime(df_temp['일시'], format='%Y-%m-%d', errors='coerce')
start_month_day = '03-15'
end_month_day = '09-10'

# 연도별 생육기 기간 필터링
df_temp = df_temp[
    df_temp['일시'].apply(
        lambda x: pd.to_datetime(f"{x.year}-{start_month_day}") <= x <= pd.to_datetime(f"{x.year}-{end_month_day}")
    )
]

# 3. 연도, 시도별 최고기온 계산
df_temp['year'] = df_temp['일시'].dt.year
highest_temp = df_temp.groupby(['year', '시도명'])['최고기온(°C)'].max().reset_index(name='생육기_최고기온(°C)')


# 4. 세종특별자치시 데이터 대체 및 추가

# 4-1. 2018년 이후 세종 데이터는 '세종특별자치시'로 이미 존재할 수 있으므로,
# '대전광역시'의 지점명을 매핑받은 '세종특별자치시'의 최고기온이 있는지 확인
# (세종 지점명이 '세종'으로 매핑되었을 경우)

# 기존에 계산된 최고기온 데이터프레임 (highest_temp)에서 '세종특별자치시' 데이터는 
# 기상 관측소 '세종'에서 온 것이므로 그대로 사용합니다.
# 다만, 2018년의 경우 데이터에 중복 행이 들어간 것으로 보였으므로, 
# '세종특별자치시' 데이터가 이미 있으면 추가하지 않습니다.

# 2015, 2016, 2017년 세종특별자치시의 데이터 프레임 생성
sejong_to_add = []
for year in [2015, 2016, 2017]:
    # 해당 연도의 대전 최고기온 값 찾기
    daejeon_temp = highest_temp[
        (highest_temp['year'] == year) & (highest_temp['시도명'] == '대전광역시')
    ]['생육기_최고기온(°C)']
    
    if not daejeon_temp.empty:
        # 데이터가 있으면 세종 데이터로 추가
        sejong_data = {
            'year': year,
            '시도명': '세종특별자치시',
            '생육기_최고기온(°C)': daejeon_temp.values[0]
        }
        sejong_to_add.append(sejong_data)

# 2018년 세종 데이터 추가 (단, 이미 데이터가 없는 경우에만)
year_2018 = 2018
is_sejong_2018_exists = '세종특별자치시' in highest_temp[highest_temp['year'] == year_2018]['시도명'].values

if not is_sejong_2018_exists:
    # 2018년 세종 데이터가 없다면, 대전 데이터로 대체하여 추가
    daejeon_temp_2018 = highest_temp[
        (highest_temp['year'] == year_2018) & (highest_temp['시도명'] == '대전광역시')
    ]['생육기_최고기온(°C)']
    
    if not daejeon_temp_2018.empty:
        sejong_data_2018 = {
            'year': year_2018,
            '시도명': '세종특별자치시',
            '생육기_최고기온(°C)': daejeon_temp_2018.values[0]
        }
        sejong_to_add.append(sejong_data_2018)


# 새로운 세종 데이터를 기존 데이터에 추가
if sejong_to_add:
    highest_temp = pd.concat([highest_temp, pd.DataFrame(sejong_to_add)], ignore_index=True)


# 5. 최종 정렬 및 저장
highest_temp = highest_temp.sort_values(by=['year', '시도명']).reset_index(drop=True)

# 시도별_생육기_최고기온.csv 파일 저장
highest_temp.to_csv('시도별_생육기_최고기온.csv', index=False, encoding='utf-8')

print("✅ 시도별 생육기 최고기온 데이터 처리가 완료되었습니다.")
print("✅ 2015년, 2016년, 2017년 세종특별자치시 데이터가 대전광역시 최고기온으로 대체되었습니다.")
print("✅ 2018년 세종특별자치시 데이터는 중복을 방지하여 추가되었습니다.")

✅ 시도별 생육기 최고기온 데이터 처리가 완료되었습니다.
✅ 2015년, 2016년, 2017년 세종특별자치시 데이터가 대전광역시 최고기온으로 대체되었습니다.
✅ 2018년 세종특별자치시 데이터는 중복을 방지하여 추가되었습니다.
